[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [27]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np

In [2]:
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 972kB 7.2MB/s 
     |████████████████████████████████| 6.5MB 15.0MB/s 
     |████████████████████████████████| 14.8MB 261kB/s 


# Data Preparation

In [3]:
if os.path.exists('PGM-Project'):
  !git -C PGM-Project/ pull 
else:
  u = input('GitHub username: ')
  p = getpass('GitHub password: ')
  !git clone https://$u:$p@github.com/remidion/PGM-Project.git
  del u, p

GitHub username: learicard
GitHub password: ··········
Cloning into 'PGM-Project'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 14), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


## Load toy data

In [4]:
planned, d_2016_10_21 = pd.read_excel("PGM-Project/data/toy.xlsx", sheet_name=[0,1]).values()

## Download MBTA Data

In [5]:
if not os.path.exists("MBTA Bus Ridership.geojson"):
  !wget https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
  !mv eec03d901d2e470ebd5758c60d793e8e_0.geojson "MBTA Bus Ridership.geojson"


--2020-11-20 16:07:58--  https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
Resolving opendata.arcgis.com (opendata.arcgis.com)... 34.207.11.89, 34.233.182.150, 3.224.48.55, ...
Connecting to opendata.arcgis.com (opendata.arcgis.com)|34.207.11.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘eec03d901d2e470ebd5758c60d793e8e_0.geojson’

eec03d901d2e470ebd5     [          <=>       ]   1.29G  45.5MB/s    in 30s     

2020-11-20 16:08:29 (43.5 MB/s) - ‘eec03d901d2e470ebd5758c60d793e8e_0.geojson’ saved [1380620043]



In [6]:
if not os.path.exists("data"):
  !wget https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
if not os.path.exists("MBTA Bus Arrival Departure Aug-Sept 2018.csv"):
  !unzip data

if not os.path.exists("data.1"):
  !wget https://www.arcgis.com/sharing/rest/content/items/1bd340b39942438685d8dcdfe3f26d1a/data
if not os.path.exists("MBTA Bus Arrival Departure Apr-June 2019.csv"):
  !unzip data.1

--2020-11-20 16:08:39--  https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
Resolving www.arcgis.com (www.arcgis.com)... 34.199.206.244, 18.234.22.27, 18.234.22.251, ...
Connecting to www.arcgis.com (www.arcgis.com)|34.199.206.244|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ago-item-storage.s3.us-east-1.amazonaws.com/d685ba39d9a54d908f49a2a762a9eb47/MBTA_Bus_Arrival_Departure_Times_2018.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEAAaCXVzLWVhc3QtMSJIMEYCIQDjNhVAofEjIOo0GSxQqI2aJHy%2FXYXmY0SZWdInEAfbKwIhAMjeAMDDLBRQRj3622Qy5tsqycovQZtOLphtjaPhV%2F3WKrQDCHkQABoMNjA0NzU4MTAyNjY1IgyqXouOibjk3%2BtvR%2BIqkQMm6X3UTGvh5S%2BPwXxjQ%2FmlHPSc03WhhNaNjv9aALWLSfqs6G%2FEiGEBNcAAEWFylWQ7Bta%2FZUEOviUhR0fRBfe5ersJ5xBleUaveiiwwoL893NPFQOBtvhIE1um8KkBe9IYGBemPiLPbgq0DX44lN0yqGOaV3zAT9NxiMQuxvqPfizc%2Bq3%2B00FtBqemKvwPzr9OP%2FjLWpRyaSdPf6gDPFP5IsWOFNR6s7YWeeWfPjK3OJOj0j8tk08ZCcGebuz%2FQ%2FWu%2ByU8X%2B9YT9FeSBWNtb2l5W6o3FZCzXI4uAL

## Load MBTA Data

### MBTA Bus Ridership

In [7]:
df = gpd.read_file("MBTA Bus Ridership.geojson")

In [8]:
df

,FID,season,route_id,route_variant,direction_id,trip_start_time,day_type_id,day_type_name,stop_name,stop_id,stop_sequence,boardings,alightings,load_,sample_size,geometry
0,1,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,WASHINGTON ST OPP RUGGLES ST,1,2,0.4,0.3,7.8,13,None
1,2,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,ALBANY ST OPP RANDALL ST,10003,5,0.1,0.0,9.4,13,None
2,3,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ SIDNEY ST,101,19,0.0,0.4,10.4,13,None
3,4,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ PROSPECT,102,20,0.6,2.5,8.5,13,None
4,5,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ BIGELOW S,104,21,0.1,0.2,8.4,13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451727,3451728,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,HIGHLAND AVE @ PINE ST,5062,9,0.0,0.0,0.0,5,None
3451728,3451729,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,CLIFTON ST @ DEXTER ST,5066,15,0.0,0.0,0.0,5,None
3451729,3451730,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,CLIFTON ST @ CEDAR ST,5067,16,0.0,0.0,0.0,5,None
3451730,3451731,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,SUMMER ST @ CLIFTON ST,5068,17,0.0,0.0,0.0,5,None


### MBTA Bus Arrival

In [9]:
df2 = pd.read_csv("MBTA Bus Arrival Departure Apr-June 2019.csv")

In [10]:
df2

,service_date,route_id,direction,half_trip_id,stop_id,stop_name,stop_sequence,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2019-04-01,01,Inbound,42976988.0,75,mit,4.0,Midpoint,Schedule,1900-01-01 05:19:00,1900-01-01 05:21:20,NaN,NaN
1,2019-04-01,01,Inbound,42976988.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01 05:23:00,1900-01-01 05:24:17,NaN,NaN
2,2019-04-01,01,Inbound,42976988.0,187,masta,6.0,Midpoint,Schedule,1900-01-01 05:25:00,1900-01-01 05:26:05,NaN,NaN
3,2019-04-01,01,Inbound,42976988.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01 05:29:00,1900-01-01 05:27:59,NaN,NaN
4,2019-04-01,01,Inbound,42977170.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01 05:30:00,1900-01-01 05:29:55,1200.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670643,2019-06-30,SL5,Outbound,44045095.0,15176,mawor,8.0,Midpoint,Headway,1900-01-02 00:54:00,NaN,900.0,NaN
7670644,2019-06-30,SL5,Outbound,44045095.0,55,Wasma,9.0,Midpoint,Headway,1900-01-02 00:55:00,NaN,900.0,NaN
7670645,2019-06-30,SL5,Outbound,44045095.0,60,Walen,10.0,Midpoint,Headway,1900-01-02 00:56:00,NaN,900.0,NaN
7670646,2019-06-30,SL5,Outbound,44045095.0,61,Melwa,11.0,Midpoint,Headway,1900-01-02 00:57:00,NaN,900.0,NaN


## Selection of arrival bus route

In [11]:
from operator import add
import glob

del df2

bus_arrival_files = glob.glob('./*.csv')

df_cumulative = pd.DataFrame()
for f in bus_arrival_files:
  df_tmp = pd.read_csv(f)
  df_cumulative = df_cumulative.append(df_tmp[['route_id','direction','stop_id']])
  print(f"Loaded {f}")

del df_tmp

Loaded ./MBTA Bus Arrival Departure Apr-June 2019.csv
Loaded ./MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded ./MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded ./MBTA Bus Arrival Departure Oct-Dec 2019.csv
Loaded ./MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded ./MBTA Bus Arrival Departure Jan-Mar 2019.csv


In [14]:
# Calculate visits per stop
group = df_cumulative[['route_id','direction','stop_id']].groupby(['route_id','direction'])
unique_stops = group.nunique().rename(columns={"stop_id":"unique_stop_ids"})
total_stops = pd.DataFrame(group.size()).rename(columns={0:"total_stops"})
stops = pd.merge(unique_stops, total_stops, left_index=True, right_index=True)
stops['visits_per_stop'] = stops['total_stops'] / stops['unique_stop_ids']

In [18]:
stops.sort_values('visits_per_stop', ascending=False).head(50)

unique_stop_ids  total_stops  visits_per_stop
route_id direction                                               
SL5      Outbound                12       814194     67849.500000
         Inbound                 13       872944     67149.538462
111      Inbound                 12       777438     64786.500000
SL1      Outbound                 7       448292     64041.714286
31       Outbound                 5       304750     60950.000000
         Inbound                  5       300727     60145.400000
SL2      Outbound                 6       345890     57648.333333
         Inbound                  6       335191     55865.166667
111      Outbound                13       718718     55286.000000
23       Inbound                 10       549635     54963.500000
SL1      Inbound                 13       709980     54613.846154
01       Outbound                 9       488393     54265.888889
         Inbound                  9       472691     52521.222222
39       Inbound                 11       561718     51065.272727
28       Inbound                 11       552235     50203.181818
39       Outbound                10       496903     49690.300000
743      Outbound                11       544599     49509.000000
77       Outbound                11       538967     48997.000000
         Inbound                 11       536599     48781.727273
23       Outbound                12       548555     45712.916667
73       Inbound                  8       337072     42134.000000
         Outbound                 8       333680     41710.000000
743      Inbound                 13       541382     41644.769231
SL4      Inbound                 12       488522     40710.166667
57       Outbound                11       429409     39037.181818
SL4      Outbound                12       447846     37320.500000
28       Outbound                16       531276     33204.750000
71       Inbound                  9       292731     32525.666667
         Outbound                 9       292047     32449.666667
746_     Inbound                  4       129754     32438.500000
11       Inbound                  8       255860     31982.500000
07       Inbound                  6       185578     30929.666667
15       Outbound                10       291775     29177.500000
22       Inbound                 14       399692     28549.428571
57       Inbound                 15       426974     28464.933333
746_     Outbound                 4       111214     27803.500000
116      Outbound                 6       163511     27251.833333
93       Outbound                 6       156767     26127.833333
07       Outbound                 6       155275     25879.166667
15       Inbound                 12       307163     25596.916667
44       Inbound                  8       204572     25571.500000
42       Inbound                  4       101804     25451.000000
         Outbound                 4       101787     25446.750000
32       Outbound                16       402072     25129.500000
         Inbound                 17       425293     25017.235294
66       Inbound                 17       422853     24873.705882
88       Outbound                 7       172868     24695.428571
44       Outbound                 8       196977     24622.125000
69       Outbound                 4        98021     24505.250000
22       Outbound                16       389951     24371.937500

## Selection of ridership bus route

In [17]:
tot_sample_size = df.groupby(by=['route_id', 'direction_id'], as_index=False)['sample_size'].sum()
tot_sample_size.sort_values(by = 'sample_size', ascending = False).head(50)

,route_id,direction_id,sample_size
94,23,1,857938
112,28,1,827974
1,1,1,758696
248,66,1,754626
120,32,1,746150
93,23,0,735042
111,28,0,724153
84,22,1,703984
119,32,0,667073
0,1,0,649238


In [33]:
df['route_id'].unique()

array(['1', '10', '100', '101', '104', '105', '106', '108', '109', '11',
       '110', '111', '112', '114', '116', '117', '119', '120', '121',
       '131', '132', '134', '136', '137', '14', '15', '16', '17', '170',
       '171', '18', '19', '201', '202', '21', '210', '211', '212', '214',
       '215', '216', '217', '22', '220', '221', '222', '225', '23', '230',
       '236', '238', '24', '240', '245', '26', '27', '28', '29', '30',
       '31', '32', '325', '326', '33', '34', '34E', '35', '350', '351',
       '352', '354', '36', '37', '38', '39', '4', '40', '41', '411', '42',
       '424', '426', '428', '429', '43', '430', '434', '435', '436',
       '439', '44', '441', '442', '448', '449', '45', '450', '451', '455',
       '456', '459', '465', '47', '5', '50', '501', '502', '503', '504',
       '505', '51', '52', '55', '553', '554', '556', '558', '57', '57A',
       '59', '60', '62', '64', '65', '66', '67', '68', '69', '7', '70',
       '701', '708', '70A', '71', '72', '73', '74', '74

# Preprocess data

## Toy 

In [ ]:
planned.columns

Index(['Ligne', 'De', 'Début', 'Fin', 'À', 'NoPermanent', 'Voiture', 'Type',
       'Direction'],
      dtype='object')

In [ ]:
filter_rows = (planned['Ligne'] == 10.0) & \
              (planned['Direction'] == "Nord")

filter_columns = ['Ligne', 'De', 'Début', 'Fin', 'À', 'NoPermanent', 'Direction']

planned[filter_rows][filter_columns]

,Ligne,De,Début,Fin,À,NoPermanent,Direction
2,10.0,M15001,2000-01-01 20:21:00,2000-01-01 17:39:00,H052,25286880,Nord
4,10.0,M15001,2000-01-01 22:21:00,2000-01-01 16:18:00,H052,80018209,Nord
6,10.0,M15001,2000-01-02 00:21:00,2000-01-01 14:55:00,H052,79994754,Nord
17,10.0,M15001,2000-01-02 11:21:00,2000-01-01 08:12:00,H052,25286806,Nord
19,10.0,M15001,2000-01-02 13:21:00,2000-01-01 07:02:00,H052,33657803,Nord
21,10.0,M15001,2000-01-02 15:21:00,2000-01-01 05:59:00,H052,25286802,Nord


In [ ]:
filter_rows = (d_2016_10_21['ROUTE_ID'] == 10) & \
              (d_2016_10_21['DIRECTION'] == 1)

filter_columns = ['MEA_DATE', 'TRIP_ID', 'ROUTE_ID',
       'DIRECTION', 'STOP_ID', 'SCHEDULED_TIME',
       'MEA_ARR_TIME', 'MEA_DEP_TIME', 'NB_BOARDING', 'NB_DEBARKING',
       'PASSENGER_LOAD', 'AVG_PASSENGER_LOAD_TRIP', 'SYSRECNO']

d_2016_10_21[filter_rows][filter_columns]

,MEA_DATE,TRIP_ID,ROUTE_ID,DIRECTION,STOP_ID,SCHEDULED_TIME,MEA_ARR_TIME,MEA_DEP_TIME,NB_BOARDING,NB_DEBARKING,PASSENGER_LOAD,AVG_PASSENGER_LOAD_TRIP,SYSRECNO
0,42664,23,10,1,120263,1092600,1092790,1093000,0,16,0,36.0,101471030
1,42664,23,10,1,108481,1073400,1066570,1073500,1,0,1,NaN,101471031
4,42664,21,10,1,120263,1042200,1040910,1050880,0,12,0,39.0,101471034
5,42664,21,10,1,108481,1021200,1017790,1021520,0,0,0,NaN,101471035
8,42664,19,10,1,120263,991800,988900,1001040,0,20,0,41.0,101471038
9,42664,19,10,1,108481,970800,971500,972090,0,0,0,NaN,101471039
12,42664,8,10,1,120263,753600,752210,752370,0,12,0,36.0,101471042
13,42664,8,10,1,108481,732600,726450,732740,4,0,4,NaN,101471043
16,42664,6,10,1,120263,706800,708860,710930,0,26,0,51.0,101471046
17,42664,6,10,1,108481,686400,686530,687930,0,0,0,NaN,101471047
